In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import fasttext.util
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
muse_path='/content/drive/MyDrive/sarvam_project/word_translation.txt'

In [ ]:
# Load FastText models
model_en = fasttext.load_model('/content/drive/MyDrive/sarvam_project/cc.en.300.bin')
all_en_matrix=[model_en.get_word_vector(word) for word in model_en.words[:1000000]]
all_en_matrix2=[model_en.get_word_vector(word) for word in model_en.words[1000000:]]
all_en_matrix.extend(all_en_matrix2)
del model_en
gc.collect()


47

In [ ]:
import numpy as np
np.save('/content/drive/MyDrive/sarvam_project/all_en_matrix.npy',np.array(all_en_matrix))
del all_en_matrix
gc.collect()

0

In [ ]:
# Load FastText models
model_hi = fasttext.load_model('/content/drive/MyDrive/sarvam_project/cc.hi.300.bin')
all_hin_matrix=[model_hi.get_word_vector(word) for word in model_hi.words[:1000000]]



In [ ]:
all_hin_matrix2=[model_hi.get_word_vector(word) for word in model_hi.words[1000000:]]

In [ ]:
all_hin_matrix.extend(all_hin_matrix2)

In [ ]:
len(all_hin_matrix[0])

300

In [ ]:


del model_hi
gc.collect()

0

In [ ]:
np.save('/content/drive/MyDrive/sarvam_project/all_hni_matrix.npy',np.array(all_hin_matrix))
del all_hin_matrix
gc.collect()

0

In [ ]:
import requests
import chardet

# URL of the text file
url = 'https://dl.fbaipublicfiles.com/arrival/dictionaries/hi-en.0-5000.txt'

# Fetch the content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Detect the encoding of the content
    detected_encoding = chardet.detect(response.content)['encoding']
    print(f"Detected encoding: {detected_encoding}")

    # Decode the content using the detected encoding
    text_content = response.content.decode(detected_encoding)

    # Store the content in a local text file with UTF-8 encoding
    with open('/content/drive/MyDrive/sarvam_project/word_translation.txt', 'w', encoding='utf-8') as file:
        file.write(text_content)

    print("File has been saved")
else:
    print("Failed to retrieve the file")


Detected encoding: utf-8
File has been saved


In [ ]:
# Load FastText models
model_en = fasttext.load_model('/content/drive/MyDrive/sarvam_project/cc.en.300.bin')

In [ ]:
!touch /content/drive/MyDrive/sarvam_project/list_en_words.txt
with open('/content/drive/MyDrive/sarvam_project/list_en_words.txt','w') as f:
  for item in model_en.words:
    f.write(f'{item}\n')


In [ ]:
del model_en
gc.collect()

In [ ]:
model_hi = fasttext.load_model('/content/drive/MyDrive/sarvam_project/cc.hi.300.bin')

In [ ]:
!touch /content/drive/MyDrive/sarvam_project/list_hi_words.txt
with open('/content/drive/MyDrive/sarvam_project/list_hi_words.txt','w') as f:
  for item in model_hi.words:
    f.write(f'{item}\n')

In [ ]:
del model_hi
gc.collect()

0

In [ ]:


with open('/content/drive/MyDrive/sarvam_project/list_en_words.txt') as f:
    list_en_words = [line.strip() for line in f.readlines()]
with open('/content/drive/MyDrive/sarvam_project/list_hi_words.txt') as f:
    list_hi_words = [line.strip() for line in f.readlines()]



In [ ]:
type(list_hi_words[0])

str

In [ ]:
!touch /content/drive/MyDrive/sarvam_project/trunc_lexicons.txt
word_pairs=[]
# Load bilingual lexicon
with open('/content/drive/MyDrive/sarvam_project/word_translation.txt') as f:
  i=0
  for line in f.readlines():
    x,y=line.strip().split()
    if(i%500==0):
      print(i)
    i+=1
    if x in list_hi_words and y in list_en_words:
      word_pairs.append([x,y])




0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000


In [ ]:
print(len(word_pairs))

7910


In [ ]:
with open("/content/drive/MyDrive/sarvam_project/trunc_lexicons.txt","w") as f:
  for pair in word_pairs:
    f.write(f"{pair[0]} {pair[1]}\n")

In [ ]:
# Load FastText models
model_en = fasttext.load_model('/content/drive/MyDrive/sarvam_project/cc.en.300.bin')

In [ ]:

en_embeddings = [model_en.get_word_vector(pair[1]) for pair in word_pairs]
del model_en
gc.collect()



In [ ]:
model_hi = fasttext.load_model('/content/drive/MyDrive/sarvam_project/cc.hi.300.bin')

In [ ]:

hi_embeddings = [model_hi.get_word_vector(pair[0]) for pair in word_pairs]
del model_hi
gc.collect()



In [ ]:
print(len(hi_embeddings[0]))

In [ ]:
import numpy as np
from sklearn.utils.extmath import randomized_svd

# Convert lists to numpy arrays
en_matrix = np.array(en_embeddings)
hi_matrix = np.array(hi_embeddings)
print(en_matrix.shape)
print(hi_matrix.shape)
# Procrustes alignment
def orthogonal_procrustes(X, Y):
    U, _, Vt = np.linalg.svd(np.dot(X.T, Y))
    return np.dot(U, Vt)

# Find optimal orthogonal transformation matrix
W = orthogonal_procrustes(en_matrix, hi_matrix)




In [ ]:
all_en_matrix=np.load('/content/drive/MyDrive/sarvam_project/all_en_matrix.npy')
# Map English embeddings to Hindi space
en_aligned = np.dot(all_en_matrix, W)

In [ ]:
print((en_aligned).shape)
print(hi_matrix.shape)


In [ ]:
del all_en_matrix
gc.collect()

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Function to find the nearest neighbor based on cosine similarity
# def find_nearest_neighbor(embedding, target_matrix):
#     similarities = cosine_similarity([embedding], target_matrix)[0]
#     nearest_neighbor_idx = np.argmax(similarities)
#     return nearest_neighbor_idx, similarities[nearest_neighbor_idx]

# # Verify alignment quality by comparing aligned English words with Hindi translations
# correct_matches = 0
# total_pairs = hi_matrix.shape[0]

# for i in range(total_pairs):
#     index_find=list_en_words.index(word_pairs[i][1])
#     aligned_embedding = en_aligned[index_find]
#     expected_hindi_embedding = hi_matrix[i]
#     #print(aligned_embedding.shape,expected_hindi_embedding.shape)

#     # Find the nearest neighbor for the aligned English word in the Hindi embedding space
#     nearest_neighbor_idx, similarity_score = find_nearest_neighbor(aligned_embedding, hi_matrix)

#     # Compare the found nearest neighbor with the expected Hindi word
#     if (nearest_neighbor_idx==i):
#         correct_matches += 1

# # Compute accuracy
# accuracy = correct_matches / total_pairs
# print(f"Alignment Accuracy while training: {accuracy * 100:.2f}%")


In [ ]:
with open('/content/drive/MyDrive/sarvam_project/word_translation_test.txt') as f:
    test_word_pairs = [line.strip().split() for line in f.readlines()]

In [ ]:
all_hi_matrix=np.load('/content/drive/MyDrive/sarvam_project/all_hni_matrix.npy')

In [ ]:
import numpy as np
import faiss  # Facebook's ANN library for efficient similarity search

# Preprocess: Build an FAISS index for the Hindi matrix
d = all_hi_matrix.shape[1]  # Dimensionality of the embeddings
index = faiss.IndexFlatIP(d)  # Inner Product (for cosine similarity) index
faiss.normalize_L2(all_hi_matrix)  # Normalize vectors for cosine similarity
index.add(all_hi_matrix)  # Add the Hindi embeddings to the index

correct_matches_at_5 = 0
valid_pairs = [
    pair for pair in test_word_pairs
    if pair[1] in list_en_words and pair[0] in list_hi_words
]

for pair in valid_pairs:
    en_word, hi_word = pair[1], pair[0]
    en_idx = list_en_words.index(en_word)
    hi_idx = list_hi_words.index(hi_word)

    # Normalize the aligned embedding for cosine similarity
    aligned_embedding = en_aligned[en_idx].reshape(1, -1)
    faiss.normalize_L2(aligned_embedding)

    # Search for the k nearest neighbors
    _, nearest_neighbors_idx = index.search(aligned_embedding, 5)

    # Check if the expected Hindi index is among the neighbors
    if hi_idx in nearest_neighbors_idx[0]:
        correct_matches_at_5 += 1

# Calculate precision@k
precision_at_5 = correct_matches_at_5 / len(valid_pairs)
print(f"Precision@k (5): {precision_at_5 * 100:.2f}%")


In [ ]:

del all_hi_matrix
gc.collect()